# Numerical PDE II: Spectral Methods

## Burgers' Equation

In the previous lecture, we analyzed finite difference methods for the
linear advection equation,
\begin{align}
  \frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0,
\end{align}
and studied their stability and accuracy using von Neumann analysis
and modified equations.

Now, we turn to a nonlinear extension: the Burgers' equation.
By replacing the constant wave speed $c$ with the solution variable
itself, the equation becomes
\begin{align}
  \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0.
\end{align}
This is the inviscid Burgers' equation, a canonical model for
nonlinear advection.

In [ ]:
# Parameters

l  = 1.0   # domain size
dt = 0.01  # time step

nx = 100   # number of spatial points
nt = 20    # number of time steps

In [ ]:
import numpy as np

X, dx = np.linspace(0, l, nx, endpoint=False, retstep=True)  # spatial grid
U0    = np.sin(2*np.pi * X)  # initial condition

In [ ]:
# Forward Time Centered Space (FTCS) scheme

def FTCS(U0, dx, dt, n):
    U = [U0]
    for _ in range(n):
        U0    = U[-1]
        sigma = U0 * dt / dx
        U1    = U0 - sigma * (np.roll(U0,-1) - np.roll(U0,1)) / 2
        U.append(U1)
    return np.array(U)

In [ ]:
UFTCS = FTCS(U0, dx, dt, nt) # numerical solution

In [ ]:
from matplotlib import pyplot as plt

plt.plot(X, U0,              label="Initial Condition")
plt.plot(X, UFTCS[10], '.-', label=f'$t$={10*dt}')
plt.plot(X, UFTCS[14], '.-', label=f'$t$={14*dt}')
plt.plot(X, UFTCS[18], '.-', label=f'$t$={18*dt}')
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

In [ ]:
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML
from tqdm import tqdm

def animate(X, U):
    fig, ax = plt.subplots(1,1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    frames = []
    for n in tqdm(range(len(U))):
        f = ax.plot(X, U[n], 'C0.-', animated=True)
        frames.append(f)
        plt.close()
    
    return ArtistAnimation(fig, frames, interval=40)

In [ ]:
anim = animate(X, UFTCS)

HTML(anim.to_html5_video())  # display animation
# anim.save('FTCS.mp4')        # save animation

In [ ]:
# HANDSON: Plot other snapshots and study the oscillations.


In [ ]:
# HANDSON: Change the parameters and initial conditions.


The Burgers' equation is important because:
* It demonstrates wave steepening: smooth initial profiles evolve into
  discontinuous shocks.
* It highlights the limitations of finite difference schemes:
  dispersion and oscillations arise near shocks.

As with the linear advection equation, the FTCS scheme is
unconditionally for Burgers' equation.
Even with very small time steps, oscillations grow and the solution
eventually blows up.

The numerical solution initially appears to evolve correctly.
However, once the shock starts to form, the numerical solution quickly
develops high-frequency oscillations near the shock.
These oscillations grow exponentially in time, destroying the
solution.

There is no convergence---the method is unstable.

In [ ]:
# HANDSON: modify the upwind scheme from lecture 9 to solve the
#          Burgers' equation

def upwind(U0, dx, dt, n):
    ...

In [ ]:
dt = 0.01  # time step
nt = 100   # number of time steps

Uupwind = upwind(U0, dx, dt, nt) # numerical solution

In [ ]:
plt.plot(X, U0,                label="Initial Condition")
plt.plot(X, UFTCS[18],   '.-', label="FTCS")
plt.plot(X, Uupwind[18], '.-', label="Upwind")
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

In [ ]:
anim = animate(X, Uupwind)

HTML(anim.to_html5_video())  # display animation
# anim.save('upwind.mp4')      # save animation

In [ ]:
# HANDSON: Plot other snapshots and study the oscillations.


In [ ]:
# HANDSON: Change the parameters and initial conditions.
#          Specifically, try out dt > 0.01.


For the upwind method, the maximum `dt` that support stable solution
is limited by the Courant-Friedrichs-Lewy condition, i.e., at each
grid point, we require the Courant number $\sigma = u \Delta t/\Delta
x \leq 1$.

The scheme captures shocks robustly and monotonically.
The numerical viscosity (implicit by the numerical method) spreads
shocks over several grid cells.
However, the smooth regions damp out over time due to numerical
diffusion.

The convergence rate is first-order everywhere.

Next, let's explore the Lax-Wendroff Scheme.

In [ ]:
# Lax-Wendroff scheme

def LW(U0, dx, dt, n):
    ...

In [ ]:
dt = 0.01  # time step
nt = 100   # number of time steps

ULW = LW(U0, dx, dt, nt) # numerical solution

In [ ]:
plt.plot(X, U0,                label="Initial Condition")
plt.plot(X, UFTCS[18],   '.-', label="FTCS")
plt.plot(X, Uupwind[18], '.-', label="Upwind")
plt.plot(X, ULW[18],     '.-', label="Lax-Wendroff")
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

In [ ]:
anim = animate(X, ULW)

HTML(anim.to_html5_video())  # display animation
# anim.save('LW.mp4')      # save animation

The Lax-Wendroff scheme, similar to upwind scheme, is stable under CFL
condition.

The solution has excellent accuracy for smooth solutions.
However, it has Gibbs-like oscillations appear at shocks.
It is non-monotone but conservative.

The convergence rate is second-order for smooth flows, first-order for
discontinuous ones.

## Weak Solutions

Inviscid Burgers' equation develops discontinuities (shocks) in finite
time, even from smooth initial conditions.
For example, an initial sine wave we saw earlier steepens until the
slope blows up.

This leads to a breakdown of classical solutions.
Namely, the PDE cannot be satisfied pointwise after a discontinuity
forms.
To make sense of solutions beyond shock formation, we introduce the
concept of a weak solution.

Multiply a PDE by a smooth test function $\phi(x,t)$, integrate over
space and time, and integrate by parts:
\begin{align}
  \int_0^\infty \int_{0}^\ell
  \left(u\frac{\partial\phi}{\partial t}
        + \frac{1}{2}u^2\frac{\partial\phi}{\partial x}\right)
  dx dt = 0.
\end{align}

The solution satisfying the above equation is called a weak solution.
Note that this definition still makes sense if $u$ is discontinuous.
Thus, weak solutions allow shocks.

Not all weak solutions are physically relevant.
The entropy condition rules out unphysical "expansion shocks".
For Burgers, this requires that characteristics enter the shock (no
information leaves it).

Physical solutions must be dissipative: they do not create new
information.
Mathematically, this is enforced by requiring that solutions satisfy
an entropy inequality in addition to the PDE.

For Burgers' equation:
* Define an entropy function $\eta(u) = u^2/2$.
* Associated entropy flux: $q(u) = u^3/3$.

The entropy condition requires
\begin{align}
  \eta(u)_t + q(u)_x \le 0,
\end{align}
in the weak sense.

This inequality expresses that entropy should not decrease (no
creation of spurious "order").
For the Burgers' equation (and many fluid dynamic equations), it rules
out expansion shocks, where characteristics diverge from the shock.
Instead, it selects compressive shocks, where characteristics converge
into the shock.

## Spectral Methods

In the previous section, we introduced the concept of weak solutions
to handle shocks in Burgers' equation.
The weak formulation required us to test the PDE against smooth
functions and integrate by parts, which naturally shifted the focus
from pointwise derivatives to integral relations over function spaces.

This perspective leads directly to spectral methods.

In spectral methods, both the solution $u$ and the test functions $v$
are represented as expansions in orthogonal basis functions (such as
Fourier modes or orthogonal polynomials).
By projecting the PDE onto these basis functions, we obtain a system
of algebraic equations for the expansion coefficients.

Spectral methods are a powerful class of numerical techniques for
solving PDEs by representing the solution as a global expansion in
orthogonal basis functions.
Unlike finite difference or finite volume methods (next lecture),
which approximate derivatives locally on stencils, spectral methods
approximate the entire function globally.

For smooth solutions, this approach achieves spectral accuracy.
The error decreases faster than any power of the grid spacing, often
exponentially, as the number of modes increases.

The advantages of using spectral methods include:
1. High Accuracy (Spectral Convergence):
   * For smooth problems, spectral methods can achieve exponential
     convergence.
   * A small number of modes can resolve very fine details compared to
     high-resolution finite difference methods.
2. Efficient Computation:
   * Differentiation and integration become algebraic operations in
     spectral space.
   * With Fast Fourier Transforms (FFT), these operations are
     $\mathcal{O}(N \log N)$, making them practical for large
     problems.
3. Conservation Properties:
   * With Galerkin truncation, spectral methods naturally conserve
     invariants such as energy and enstrophy.
   * This is especially important in fluid dynamics and turbulence modeling.
4. Applications in Science and Engineering:
   * Astrophysical fluid dynamics.
   * Weather and ocean modeling.
   * Turbulence simulations.
   * Quantum mechanics and wave propagation.

### Core Ideas

We approximate a function $u(x)$ by a truncated series of orthogonal
basis functions.
For example, in the Fourier spectral method on a periodic domain:
\begin{align}
  u(x) \approx \sum_{k=-N/2}^{N/2} \hat{u}_k e^{ikx},
\end{align}
where $\hat{u}_k$ are Fourier coefficients.

Differentiation is exact in spectral space:
\begin{align}
  \frac{\partial u}{\partial x} = \sum_{k=-N/2}^{N/2} ik\hat{u}_k e^{ikx}.
\end{align}

Thus, spectral methods can be seen as a natural extension of
high-order finite differences:
instead of improving accuracy with wider local stencils, we represent
the entire solution in a global basis where differentiation is exact.

### Periodic vs. Non-Periodic Domains

* Periodic problems:
  Fourier modes $e^{ikx}$ form the natural orthogonal basis.
* Non-periodic problems:
  Chebyshev or Legendre polynomials are often used, maintaining
  spectral accuracy while accommodating boundary conditions.

### Strengths and Limitations

* Strengths:
  * Exponential convergence for smooth problems.
  * Captures global solution features with fewer degrees of freedom.
  * Efficient with FFT-based implementations.

* Limitations:
  * Require smoothness:
    discontinuities produce oscillations (Gibbs phenomenon).
  * More complex for non-periodic geometries.
  * Global coupling:
    each mode influences the entire domain, which can make very large
    systems computationally heavy.

### Spectral Galerkin vs. Pseudo-Spectral Methods

Spectral methods come in two closely related formulations.
Both begin by expanding the solution in a truncated set of orthogonal
basis functions.

The **Spectral Galerkin Method** start with the weak form of the PDE.
The governing equations are multiplied by a test function (from the
same basis set) and integrate over the domain.
The residual is then required to vanish in this weak sense:
\begin{align}
  \int \frac{\partial u}{\partial t} v dx + \int \frac{\partial f(u)}{\partial x} v dx = 0.
\end{align}
With the Fourier basis on a periodic domain, this reduces to an exact
algebraic system for the modal coefficients $\hat{u}_k$.
This approach is mathematically elegant and highlights the connection
to variational formulations.

The **Pseudo-Spectral Method**, on the other hand, compute all
derivatives in Fourier space, but evaluate nonlinear terms directly in
physical space.

The algorithm is very straightforward:
1. Transform function to Fourier space using FFT.
2. Compute derivatives in Fourier space.
3. Inverse transform the derivatives back to physical space.
4. Compute all nonlinear terms, e.g., $u\partial u/\partial x$ in
   physical space.
5. Advance grid points in time.

This avoids expensive convolutions in Fourier space, since nonlinear
products are simple in real space.
The computation complexity is $\mathcal{O}(N \log N)$ with FFT.

In this lecture, we will adopt the pseudo-spectral method to solve the
Burgers' equation.